# HOW TO AUDIT AN AI MODEL OWNED BY SOMEONE ELSE (PART 1 - USER LOG)

In this tutorial, we show how external parties can audit internal AI systems without accessing them — mitigating privacy, security, and IP costs and risks. **This tutorial uses syft 0.8.2.b0, with a datasite setup that does not use networking, to run the tutorial with networking read more in section 1.1.1**

You can read more about this tutorial and the follow up tutorials here on the [blog post](https://blog.openmined.org/).

## Model Owner Launches Stage 1 Audit Environment

**Note** : Kindly use light theme when running the demo for better visuals

In [ ]:
# install syft
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -q

In [ ]:
# third party
import pandas as pd

# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

### Launch PySyft datasite server

To start we launch a `PySyft` datasite server. This is the backend that stores the private data.

In [ ]:
server = sy.orchestra.launch(name="syft-datasite", reset=True)

There are 3 ways to launch a `PySyft` datasite

**A) From a notebook, with simulated networking \*\*THIS NOTEBOOK\*\***
  - Apart from the network calls, this uses exactly the same code as other setups
  - run orchestra **without a port**: `sy.orchestra.launch(name="syft-datasite")`
  
**B) From a notebook with networking (also supports docker)**
  - This spawns a separate process that starts a uvicorn webserver
  - run orchestra **with a port**:`sy.orchestra.launch(name="syft-datasite", port=8080)`
  
**C) From the command line (supports docker/kubernetes)**
  - setup for production
  - run `syft launch`  from the terminal
  
  
We are using the **A)** here, as it is the only option available using google colab, switching to a real webserver is as easy as running this notebook in jupyter locally and adding a port. Read more about deployment on our [README.md](https://github.com/OpenMined/PySyft) and other setups for syft [here](https://github.com/OpenMined/PySyft/tree/dev/notebooks/tutorials/data-engineer)

### Login


We can now login to our datasite using the default admin credentials. In production we would change these.

In [ ]:
mo_client = server.login(email="info@openmined.org", password="changethis")

### Configure server to allow user registration

For this tutorial we allow other users to create their own account. New accounts will get limited permissions and will only be able to see the mock version of any datasets we upload to the datasite.

In [ ]:
mo_client.settings.allow_guest_signup(enable=True)

## Model Owner Uploads What will be Audited

We are ready to create a dataset. Our dataset consists of prompts that were used as input for our language model, and their corresponding continuations. For example, in the first row we see that the `prompt` for the model was *"Jacob Zachar is an American actor whose"*, and the `result` was "*erythemal body image makes him look like an infant in the bedroom.*"  We also have a mock version of the same dataset. The mock dataframe contains no meaningful data, but it has the same columns, size and datatypes as the real data.

In [ ]:
dataset_url = "https://github.com/OpenMined/datasets/raw/main/AuditingBlogpost"
model_log = pd.read_csv(f"{dataset_url}/gpt2_100row.csv")
mock_model_log = pd.read_csv(f"{dataset_url}/gpt2_100row_mock.csv")

In [ ]:
model_log

In [ ]:
mock_model_log

To upload our dataset to the datasite we need to wrap it in a `Syft Dataset` object. We can add some metadata to the object.

In [ ]:
main_contributor = sy.Contributor(
    name="Jeffrey Salazar", role="Dataset Creator", email="jsala@ailab.com"
)

gpt2_user_log = sy.Dataset(
    name="GPT-2 Activity Log",
    description="User interactions from GPT-2 usage in text completion.",
    contributors=[main_contributor],
    asset_list=[
        sy.Asset(
            name="gpt2-mar23-prompts-responses",
            description="Text prompts and corresponding model predictions from GPT-2 (March 2023)",
            contributors=[main_contributor],
            data=model_log,
            mock=mock_model_log,
        )
    ],
)

In [ ]:
mo_client.upload_dataset(gpt2_user_log)

This was the bulk of the work for the Model owner, its the auditors turn now to propose a project.

## Auditor Creates Account and Proposes Project


We first create an account and login.

In [ ]:
auditor_client = server.register(
    name="Peter Jones",
    email="pjones@aisb.org",
    password="password1234",
    password_verify="password1234",
)
auditor_client = server.login(email="pjones@aisb.org", password="password1234")

Our account has limited permissions, but we are able to access the mock part of the dataset to code against. 

In [ ]:
dataset = auditor_client.datasets[0]
asset = dataset.assets[0]
asset

In [ ]:
mock = asset.mock
mock

We can now create a `Syft Project` which will act as a wrapper for all the requests on this `Dataset`

In [ ]:
audit_project = sy.Project(
    name="Model Output Audit",
    description="Auditing GPT2 model outputs for toxicity, bias, etc.",
    members=[auditor_client],
)
audit_project

Before we submit our actual audit code, we need to write the code. Writing code without input is often quite challenging and therefore we use the mock to write our code. Once we verified that everything works and we have no errors, we can submit the code for approval.

In [ ]:
# third party
import evaluate

toxicity = evaluate.load("toxicity", module_type="measurement")
indices, inputs = mock.id.tolist(), mock["result"].tolist()
toxicity_results = toxicity.compute(predictions=inputs)
mock_result = pd.DataFrame(
    toxicity_results["toxicity"], index=indices, columns=["toxicity"]
)

In [ ]:
mock_result

With that set up, we are ready to write the code that we want to execute on the dataset. We do this by writing a function and wrapping that function with a `@sy.syft_function` decorator, this particular decorator requests that we can run this function exactly once on the dataset that was just uploaded. Within the function we compute and return the toxicity scores for the results of the model.

In [ ]:
@sy.syft_function_single_use(data=dataset.assets[0])
def model_output_analysis(data):
    """
    Evaluate the model's quantify the toxicity of the input texts using the R4 Target Model,
    a pretrained hate speech classification model
    Evaluate the model's estimated language polarity towards and social perceptions of a demographic
    (e.g. gender, race, sexual orientation).
    """
    # third party
    import evaluate
    import pandas as pd

    toxicity = evaluate.load("toxicity", module_type="measurement")
    indices, inputs = data.id.tolist(), data["result"].tolist()
    toxicity_results = toxicity.compute(predictions=inputs)
    return pd.DataFrame(
        toxicity_results["toxicity"], index=indices, columns=["toxicity"]
    )

We can now request code execution of our function by calling the `.create_code_request` method

In [ ]:
audit_project.create_code_request(model_output_analysis, auditor_client)

We can inspect our code submission, which means we now have to wait for approval from the model owner.

In [ ]:
auditor_client.code

As a last step we start out project, and we switch back to the perspective of the model owner.

In [ ]:
project = audit_project.send()
project

## Model Owner Reviews Proposed Project

Now that the model owner has a new incoming request, the goal is to approve or deny the request based on the code. This may include running the code on mock data first or asking questions to the auditor. In our case we will simply review the code and approve it.

In [ ]:
mo_client.projects

Lets view the newly created project

In [ ]:
project = mo_client.projects[0]
project

And now view the corresponding request

In [ ]:
request = mo_client.requests[0]
request

We can view the code to review it

In [ ]:
request.code

Once the model owner feels confident that this code is not malicious, we can run the function on the real data to inspect the result.

In [ ]:
asset = mo_client.datasets[0].assets[0]

In [ ]:
real_result = request.code.run(data=asset.data)
real_result

If everything looks good, we can approve the request

In [ ]:
request.approve()

## Auditor computes Final Results

In [ ]:
asset = auditor_client.datasets[0].assets[0]
result = auditor_client.code.model_output_analysis(data=asset).get()

In [ ]:
result

👏 Tutorial Complete, you can read more about PySyft on the accompanying [blog post](https://blog.openmined.org/) or on our GitHub [README.md](https://github.com/OpenMined/pysyft)

Share this Colab Notebook:<br />
<a href="http://colab.research.google.com/github/OpenMined/PySyft/blob/dev/notebooks/tutorials/model-auditing/colab/01-user-log.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" />
</a>